# Evolution of agents - multi-agent

Agents working together on a single task.
<img src="./images/multi_agent.png" alt="Agent with Actions" style="max-height: 300px;" />

**Description:**
This notebook explores multi-agent orchestration, where several agents collaborate to solve complex tasks. It demonstrates agent-to-agent communication, coordination, and division of labor, showing how specialized agents can work together for more powerful solutions.

In [ ]:
# Example: Inference using Semantic Kernel
from semantic_kernel import Kernel
import os
from setup import (
    get_project_client,
    create_agent,
    create_weather_openapi_tool,
    get_connection_by_name,
    test_agent,
)
from AzureStandardLogicAppTool import create_logic_app_tools
from azure.ai.agents.models import BingGroundingTool
from azure.ai.agents.models import ConnectedAgentTool

client = await get_project_client()

logic_app_tools = create_logic_app_tools(
    logic_app_subscription_id=os.environ.get("LOGIC_APP_SUBSCRIPTION_ID"),
    logic_app_resource_group=os.environ.get("LOGIC_APP_RESOURCE_GROUP"),
    logic_app_name=os.environ.get("LOGIC_APP_NAME"),
    foundry_subscription_id=os.environ.get("AZURE_AI_FOUNDRY_SUBSCRIPTION_ID"),
    foundry_resource_group=os.environ.get("AZURE_AI_FOUNDRY_RESOURCE_GROUP"),
    foundry_foundry_name=os.environ.get("AZURE_AI_FOUNDRY_NAME"),
    foundry_project_name=os.environ.get("AZURE_AI_FOUNDRY_PROJECT_NAME"),
)
current_date_tool = next(
    (tool for tool in logic_app_tools if "time" in tool.definitions[0]["openapi"].name),
    None,
)

kernel = Kernel()

office_agent = await create_agent(
    agent_name="Office365-agent",
    agent_instructions="You are a helpful assistant. You use Office 365 tools to send emails, schedule events and check calendars. Make sure you know the current date! It doesn't make sense to schedule events in the past.",
    client=client,
    kernel=kernel,
    tools=[def_ for tool in logic_app_tools for def_ in tool.definitions],
)

weather_tool = create_weather_openapi_tool()

weather_agent = await create_agent(
    agent_name="Weather-agent",
    agent_instructions="You are a helpful assistant. You use weather tools to provide weather information.",
    client=client,
    kernel=kernel,
    tools=(
        weather_tool.definitions + current_date_tool.definitions
        if current_date_tool
        else []
    ),
)

bing_grounding = BingGroundingTool(
    connection_id=await get_connection_by_name(client, "bing")
)
news_agent = await create_agent(
    agent_name="Bing-agent",
    agent_instructions="Use the Bing grounding tool to answer the user's question.",
    client=client,
    kernel=kernel,
    tools=(
        bing_grounding.definitions + current_date_tool.definitions
        if current_date_tool
        else []
    ),
)

blog_url = os.environ.get("BLOG_URL", None)
print(f"Blog URL: {blog_url}")
playwright_agent = await create_agent(
    agent_name="Playwright-agent",
    agent_instructions=f"You're a blog manager. You read blog posts and post new ones on {blog_url} using browser automation.",
    client=client,
    kernel=kernel,
    tools=[
        {
            "type": "browser_automation",
            "browser_automation": {
                "connection": {"id": await get_connection_by_name(client, "Playwright")}
            },
        }
    ],
)

connected_agents = [
    ConnectedAgentTool(
        id=office_agent.id,
        name="calendar_email_agent",
        description="""
Calendar agent can:
* check calendar of the user
* schedule new events (book running time) by providing event details (subject, start time, end time)
* email users (to, subject, body)
* check current date/time
                       """,
    ),
    ConnectedAgentTool(
        id=weather_agent.id,
        name="weather_agent",
        description="""
Weather agent can:
* provide current weather information for provided location
* provide weather forecasts for provided location
                       """,
    ),
    ConnectedAgentTool(
        id=playwright_agent.id,
        name="blog_agent",
        description=f"""
Blog agent can manage running blog {blog_url} using playwright tool
* read blog posts
* post new blog entries
                       """,
    ),
    ConnectedAgentTool(
        id=news_agent.id,
        name="news_agent",
        description="""
News agent can perform web research using bing grounding tool
                       """,
    ),
]

main_agent = await create_agent(
    agent_name="Main-agent",
    agent_instructions="""
You are **AdvisorGPT**, a helpful, professional agent supporting user queries. 
Your main objective is to fully resolve each user query related to health, fitness and running, using only verified information and the tools listed below.

---

## **Workflow**

1. **Plan Before Action:**  
   Before using any agent, briefly outline your step-by-step plan to address the user’s request.

2. **Agent Use:**  
   Use the most relevant agent(s) for accurate, up-to-date information
   If agent results are unclear or incomplete, use additional agents or ask the user for clarification.

3. **Reflect After Each Agent Call:**  
   - Did you get all the info needed?  
   - What’s the next logical step?  
   - Is further clarification needed?  
   - Is the problem fully solved?

4. **Clear Output:**  
   - Use clear sections, bullet points, numbered steps, or tables where helpful.
   - Use a joyful, helpful tone, feel free to use emojis

5. **If Unsure:**  
   - Never guess. Use Agents to find the answer or ask the user to clarify.

6. **Prohibited Topics:**  
   - If the request is outside of domain supported by connected agents, politely redirect the user:
   - Example: “I’m sorry, but I can’t discuss that topic. Is there something OTIS-related I can assist you with?”

7. **Use Date/Time Tool:**  
   - Always use `get-date when current or time-bounded information is needed.

8. **Use Location Information:**
   - Ask user for location information when relevant, especially for weather and calendar events.

9. **Use blog for memory**
   - Utilize the blog_agent check for previous posts or relevant information before scheduling new runs.

---

## **Available Agents**

| Agent Name                           | Purpose/Usage                                 | Available Tools |
|--------------------------------------|-----------------------------------------------|-----------------|
| calendar_email_agent                 | calendar management and email communication   | create_event, get_events, email_me, get_current_time |
| weather_agent                        | current weather and forecast                  | WeatherAPI, get_current_time |
| news_agent                           | bing search for current information           | bing grounding search |
| blog_agent                           | running blog management                       | Executing actions on the running blog |

**Before Using an Agent:**  
- “Let me verify your information to assist you.”  
- “One moment, I’ll check that for you.”

**After Using an Agent:**  
- “Here’s what I found: [summary]”  
- “Based on the data, here are your next steps…”

## **Key Guidelines**

- Plan first, act second.
- Reflect after every action.
- Never guess—always use tools or clarify.
- Structure responses for clarity.
- Only finish when the issue is fully resolved.

    """,
    client=client,
    kernel=kernel,
    tools=[def_ for tool in connected_agents for def_ in tool.definitions],
)

In [ ]:
thread = None
user_input = """
    I want to go for a run tomorrow in Cary, NC. Check the weather forecast and my calendar. If the weather is okay, schedule the run and post about it on my blog.  
Email me with a summary of everything you've done to piotrkarpala@microsoft.com"
"""
thread = await test_agent(client, main_agent, user_input, thread)

In [ ]:
# uncomment for follow up
# thread = await test_agent(client, main_agent, "try again", thread)